In [1]:
import matplotlib.pyplot as plt
import numpy as np 
import os 
import pandas as pd
import requests 
import json
import time 
import scipy.stats as st

#National Park Service API 

API_Key_Park = 'x4r30IhC7l6xEe3DCmQmuiefih8dcjFcPutVzRhh'
base_url = "https://api.nps.gov/api/v1/parks?stateCode="

In [2]:
import json
import requests


URL = 'https://irmaservices.nps.gov/v2/rest/unit/?unitCodes={unitcode}&format=json'
response = requests.get(URL).json()

column_names = ['FullName','UnitCode','UnitName',
                'StateCode']

state_Codes = []

national_parks = []
for park in response:
    if(park['UnitDesignationName'] == 'National Park'):
        national_parks.append([park['FullName'], 
                          park['UnitCode'], 
                          park['UnitName'],
                          park['StateCodes']])
        state_Codes.append(park['StateCodes'])
    
national_parks
np_df = pd.DataFrame(national_parks, columns=column_names)
np_df.head()

,FullName,UnitCode,UnitName,StateCode
0,Acadia National Park,ACAD,Acadia,[ME]
1,Arches National Park,ARCH,Arches,[UT]
2,Badlands National Park,BADL,Badlands,[SD]
3,Big Bend National Park,BIBE,Big Bend,[TX]
4,Biscayne National Park,BISC,Biscayne,[FL]


In [3]:
column_names = ['UnitCode','Month', 'Visitors']
query_year = "2019"
park_visitor_data = []

for unitcode in np_df['UnitCode']:
    URL = f'https://irmaservices.nps.gov/v3/rest/stats/visitation?unitCodes={unitcode}&startMonth=01&startYear={query_year}&endMonth=12&endYear={query_year}&format=json'
    response = requests.get(URL).json()
    
    for month in range(0,len(response)):
        park_visitor_data.append([unitcode, 
                          month,
                          response[month]['RecreationVisitors']])
                   
visitor_df = pd.DataFrame(park_visitor_data, columns=column_names)
visitor_df1 = visitor_df.groupby('UnitCode')
visitor_df_group = visitor_df1.sum()
visitor_df_group.head()


,Month,Visitors
UnitCode,,
ACAD,66,3437286
ARCH,66,1659702
BADL,66,970998
BIBE,66,463832
BISC,66,708522


In [4]:
Unique_Unit_Codes = visitor_df['UnitCode'].unique()
codes = np_df['UnitCode']
statec = []

unitcodes = visitor_df['UnitCode']

for i in range(len(Unique_Unit_Codes)):
    for j in range(len(codes)):
        if Unique_Unit_Codes[i] == codes[j]:
            statec.append(state_Codes[j])

In [5]:
statec1 = pd.DataFrame({'UnitCode': Unique_Unit_Codes,'State' : statec})
new = pd.merge(visitor_df_group,statec1, on = ['UnitCode'])

In [6]:
new['State'] = new['State'].apply(lambda x: tuple(x))
try1 = new.groupby('State')
try2 = try1.sum()
park_num = []
for i in range(len(try2)):
    park_num.append(try2['Month'][i] / 66)

In [7]:
try2['Number of Parks'] = park_num

In [8]:
from census import Census
from us import states
c = Census("f02ff3c3e214fc307eb6c109357fb47176ed13e1")
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E"), {'for': 'state:*'})



#Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "Name", "state": "State"})

census_pd1 = census_pd[['Name','Population','State']]
census_pd1.head()

,Name,Population,State
0,Puerto Rico,3468963.0,72
1,Alabama,4850771.0,01
2,Alaska,738565.0,02
3,Arizona,6809946.0,04
4,Arkansas,2977944.0,05


In [9]:
#States = ['TX','NM','AZ','OK','LA','AR','WV','KY','TN','NC','SC','GA','AL','MS','FL']
States = ["AK","AL","AR","AZ","CA","CO","CT","DC","DE",
                          "FL","GA","GU","HI","IA","ID", "IL","IN","KS","KY",
                          "LA","MA","MD","ME","MH","MI","MN","MO","MS","MT","NC",
                          "ND","NE","NH","NJ","NM","NV","NY", "OH","OK","OR","PA",
                          "PR","PW","RI","SC","SD","TN","TX","UT","VA","VI","VT","WA","WI","WV","WY"]

state_pd = pd.read_csv('states.csv')
state_pd = state_pd.rename(columns = {'State':'Name'})

New_Census = pd.merge(census_pd1,state_pd,on = ['Name'])

In [10]:
Name1 = New_Census['Abbreviation']

a = [] 
column_names1 = ['Name','Population','Abbreviation']
for i in range(len(New_Census)):
    for j in range(len(States)):
        if Name1[i] == States[j]:
            a.append([New_Census['Name'][i],
                      New_Census['Population'][i], 
                      New_Census['Abbreviation'][i]
            ])

        
a_df = pd.DataFrame(a, columns=column_names1)
a_df.head()

,Name,Population,Abbreviation
0,Alabama,4850771.0,AL
1,Alaska,738565.0,AK
2,Arizona,6809946.0,AZ
3,Arkansas,2977944.0,AR
4,California,38982847.0,CA


In [11]:
NewStates = [] 
for state in States:
    NewStates.append((f'{state}',))

maybe = [] 
column_names1 = ['Unit Code','Visitor Count','State']
for i in range(len(new)):
    for j in range(len(NewStates)):
        if new['State'][i] == NewStates[j]:
            maybe.append([new['UnitCode'][i],
                      new['Visitors'][i], 
                      new['State'][i]
            ])
    
new_df = pd.DataFrame(maybe, columns=column_names1)

In [12]:
new_df['State'] = new_df['State'].apply(lambda x: tuple(x))
group_state = new_df.groupby('State')
group_state1 = group_state.sum()
group_state1

,Visitor Count
State,
"(AK,)",372367
"(AR,)",1467153
"(AZ,)",7638225
"(CA,)",10898186
"(CO,)",5659074
"(FL,)",1906022
"(HI,)",2362770
"(IN,)",2134285
"(KY,)",551590


In [13]:
stapop = []

abbr = a_df['Abbreviation']

tt = []
for ii in range(len(new_df)):
    for kk in range(len(a_df)):
        if new_df['State'] [ii] == (f'{abbr[kk]}',):
            stapop.append([(f'{abbr[kk]}',), 
                           a_df['Population'][kk]])

In [14]:
stapop_df = pd.DataFrame(stapop,columns= ['State','Population'])
# stapop_df.drop_duplicates(
#                      keep = 'first', inplace = True) 
stapop_df1 = stapop_df.groupby('State')
stapop_df2 = stapop_df1.mean()

In [15]:
merged = pd.merge(group_state1.reset_index(),stapop_df2.reset_index(), on = ['State'])
merged

,State,Visitor Count,Population
0,"(AK,)",372367,738565.0
1,"(AR,)",1467153,2977944.0
2,"(AZ,)",7638225,6809946.0
3,"(CA,)",10898186,38982847.0
4,"(CO,)",5659074,5436519.0
5,"(FL,)",1906022,20278447.0
6,"(HI,)",2362770,1421658.0
7,"(IN,)",2134285,6614418.0
8,"(KY,)",551590,4424376.0
9,"(ME,)",3437286,1330158.0


In [49]:
import plotly.express as px
fig = px.scatter(merged, x="Population", y="Visitor Count", hover_data=["State"] )
fig.update_layout(title='Population vs. Visitor Counts of National Parks Exclusive to One State')
fig.show()

In [54]:
import plotly.express as px
fig = px.scatter(try2.reset_index(), x="Visitors", y="Number of Parks", hover_data=["State"] )
fig.update_layout(title='Vistor Count vs. Number of National Parks For Every National Park')
fig.show()